In [ ]:
async def process_att_summary(session, base_url, headers, combined_df, ums_avg_att_df):
    att_summary_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month',
                                           'ewyl-group-name', 'first-day-of-month', 'month', 'year'])
    combined_df['date'] = pd.to_datetime(combined_df['date'])    
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        student_df = combined_df[combined_df['admissions-group-name'] == student]

        first_date = pd.to_datetime(student_df['date'].min())
        first_day_current_month = first_date.replace(day=1)

        while True:
            this_month_df = student_df[(student_df['date'].dt.year == first_day_current_month.year) &
                                       (student_df['date'].dt.month == first_day_current_month.month)]

            prev_month_df = student_df[student_df['date'] < first_day_current_month]

            total_sessions_this_month = len(this_month_df)
            attended_sessions_this_month = this_month_df[this_month_df['present'] == 'Present'].count()
            attendance_percentage_this_month = get_percentage(attended_sessions_this_month, total_sessions_this_month)

            if prev_month_df.empty:
                avg_att_till_last_month = 0
            else:
                total_sessions_till_last_month = len(prev_month_df)
                attended_sessions_till_last_month = prev_month_df[prev_month_df['present'] == 'Present'].count()
                avg_att_till_last_month = get_percentage(attended_sessions_till_last_month, total_sessions_till_last_month)

            new_row = {
                'admissions-group-name': student,
                'attendance-percentage': attendance_percentage_this_month,
                'avg-att-percent-till-last-month': avg_att_till_last_month,
                'ewyl-group-name': student_df['ewyl-group-name'].iloc[0] if not student_df['ewyl-group-name'].empty else 'N/A',
                'first-day-of-month': first_day_current_month.strftime("%Y-%m-%d"),
                'month': first_day_current_month.strftime("%B"),
                'year': first_day_current_month.year
            }
            att_summary_df = pd.concat([att_summary_df, pd.DataFrame([new_row])], ignore_index=True)

            next_month_first_day = (first_day_current_month + pd.offsets.MonthBegin()).normalize()

            if student_df[student_df['date'] >= next_month_first_day].empty:
                break

            first_day_current_month = next_month_first_day

    # Conversion to JSON and Posting Data
    for index, row in att_summary_df.iterrows():
        row_id = await check_if_row_exists(row['ewyl-group-name'], row['year'], row['month'], ums_avg_att_df)

        data = row.to_dict()

        if row_id:
            await patch_attendance_summary(session, base_url, headers, row_id, data)
        else:
            await post_new_attendance_summary(session, base_url, headers, data)

    return att_summary_df

In [ ]:
async def process_att_summary(session, base_url, headers, combined_df, ums_avg_att_df):
    att_summary_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month',
                                           'ewyl-group-name', 'first-day-of-month', 'month', 'year'])
    combined_df['date'] = pd.to_datetime(combined_df['date'])    
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        student_df = combined_df[combined_df['admissions-group-name'] == student]
        # ... (rest of your loop for processing student data)

    # Now, iterate over att_summary_df to post or patch data
    for index, row in att_summary_df.iterrows():
        row_id = await check_if_row_exists(row['ewyl-group-name'], row['year'], row['month'], ums_avg_att_df)

        # Convert row to JSON-compatible format
        data = row.to_dict()
        data['first-day-of-month'] = data['first-day-of-month'].strftime("%Y-%m-%d") if data['first-day-of-month'] else None

        if row_id:
            await patch_attendance_summary(session, base_url, headers, row_id, data)
        else:
            await post_new_attendance_summary(session, base_url, headers, data)

    return att_summary_df


In [1]:
async def process_att_summary(session, base_url, headers, combined_df, ums_avg_att_df):
    att_summary_df = pd.DataFrame(columns=['admissions-group-name', 'attendance-percentage', 'avg-att-percent-till-last-month',
                                           'ewyl-group-name', 'first-day-of-month', 'month', 'year'])
    combined_df['date'] = pd.to_datetime(combined_df['date'])    
    unique_students = combined_df['admissions-group-name'].unique()

    for student in unique_students:
        student_df = combined_df[combined_df['admissions-group-name'] == student]

        # Get the earliest date in student_df for that student and set first_date as the 1st of that month
        first_date = pd.to_datetime(student_df['date'].min())
        first_day_current_month = first_date.replace(day=1)

        while True:
            this_month_df = student_df[(student_df['date'].dt.year == first_day_current_month.year) &
                                       (student_df['date'].dt.month == first_day_current_month.month)]

            prev_month_df = student_df[student_df['date'] < first_day_current_month]

            # Calculate attendance percentage for this month using get_percentage
            total_sessions_this_month = len(this_month_df)
            attended_sessions_this_month = this_month_df[this_month_df['present'] == 'Present'].count()
            attendance_percentage_this_month = get_percentage(attended_sessions_this_month, total_sessions_this_month)

            # Calculate average attendance till last month
            if prev_month_df.empty:
                avg_att_till_last_month = 0
            else:
                total_sessions_till_last_month = len(prev_month_df)
                attended_sessions_till_last_month = prev_month_df[prev_month_df['present'] == 'Present'].count()
                avg_att_till_last_month = get_percentage(attended_sessions_till_last_month, total_sessions_till_last_month)
            # Add the new row to att_summary_df
            new_row = {
                'admissions-group-name': student,
                'attendance-percentage': attendance_percentage_this_month,
                'avg-att-percent-till-last-month': avg_att_till_last_month,
                'ewyl-group-name': student_df['ewyl-group-name'].iloc[0] if not student_df['ewyl-group-name'].empty else 'N/A',
                'first-day-of-month': first_day_current_month.strftime("%Y-%m-%d"),
                'month': first_day_current_month.strftime("%B"),
                'year': first_day_current_month.year
            }
            att_summary_df = pd.concat([att_summary_df, pd.DataFrame([new_row])], ignore_index=True)
            print("attendance summary")
            print(att_summary_df)

            # Calculate the first day of the next month
            next_month_first_day = (first_day_current_month + pd.offsets.MonthBegin()).normalize()

            # Check if there is data for the next month
            if student_df[student_df['date'] >= next_month_first_day].empty:
                break  # Exit the loop if there's no data for the next month

            # Update first_day_current_month to the first day of the next month
            first_day_current_month = next_month_first_day

            for index, row in att_summary_df.iterrows():
                row_id = await check_if_row_exists(row['ewyl-group-name'], row['year'], row['month'], ums_avg_att_df)

                data = {
                    'admissions-group-name': row['admissions-group-name'],
                    'attendance-percentage': row['attendance-percentage'],
                    'avg-att-percent-till-last-month': row['avg-att-percent-till-last-month'],
                    'ewyl-group-name': row['ewyl-group-name'],  # Make sure this is a single value
                    'first-day-of-month': row['first-day-of-month'],  # Single value
                    'month': row['month'],  # Single value
                    'year': row['year']  # Single value
                }

                # Convert any Pandas Series or non-serializable types to native Python types
                for key, value in data.items():
                    if isinstance(value, pd.Series):
                        data[key] = value.iloc[0]  # Assuming you want the first element of the Series

                if row_id:
                    await patch_attendance_summary(session, base_url, headers, row_id, data)
                else:
                    await post_new_attendance_summary(session, base_url, headers, data)

    return att_summary_df